In [1]:
#Installing Packages
using Pkg
Pkg.add("InteractiveDynamics")
Pkg.add("DynamicalSystems")
Pkg.add("DifferentialEquations")
Pkg.add("Plots")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


In [2]:
#Importing Libraries
using InteractiveDynamics, DynamicalSystems, DifferentialEquations, Plots

# The Lorenz Differential Equations
The Lorenz system is a system of ordinary differential equations first studied by mathematician and meteorologist Edward Lorenz. It is notable for having chaotic solutions for certain parameter values and initial conditions. In particular, the Lorenz attractor is a set of chaotic solutions of the Lorenz system.

$$
\begin{aligned}
\dot{x} & = \sigma(y-x) \\
\dot{y} & = \rho x - y - xz \\
\dot{z} & = -\beta z + xy
\end{aligned}
$$

In [3]:
#Defining the Lorenz Differential Equations
function lorentz!(du,u,p,t)
    x,y,z = u
    sigma,rho,beta = p
    
    du[1] = sigma*(y - x)
    du[2] = rho*x - y - x*z
    du[3] = x*y - beta*z
    
    return nothing
end

lorentz! (generic function with 1 method)

In [4]:
initial = [1.0,0.0,0.0] #Initial Conditions
parameters = [10.0,28.0,8/3] #value of sigma, rho and beta
time_span = (0.0,100.0) #Time span of the equations

prob = ODEProblem(lorentz!,initial,time_span,parameters) #Defining the ODE
sol = solve(prob) #Solving the ODE

retcode: Success
Interpolation: specialized 4th order "free" interpolation, specialized 2nd order "free" stiffness-aware interpolation
t: 1264-element Vector{Float64}:
   0.0
   3.5678604836301404e-5
   0.0003924646531993154
   0.003262398390028978
   0.009058060149574726
   0.016956449405371755
   0.02768993058405869
   0.04185631256995427
   0.060240359409912465
   0.0836853459014633
   0.11336490912114593
   0.1486217173729216
   0.18703967196961463
   ⋮
  98.99459287053229
  99.0898561618104
  99.18516855260675
  99.2858697333858
  99.38350446911585
  99.46865611105598
  99.56122065843316
  99.64618558376807
  99.74364956975944
  99.83798930278873
  99.94106524965552
 100.0
u: 1264-element Vector{Vector{Float64}}:
 [1.0, 0.0, 0.0]
 [0.9996434557625105, 0.0009988049817849054, 1.7814347887992044e-8]
 [0.9961045497425811, 0.01096539972124245, 2.1469553658389054e-6]
 [0.9693592457883459, 0.08977035335005876, 0.00014380102292713885]
 [0.9242044671035574, 0.24228872786108774, 0.001046158

# Taken's Theorem (1981)

Using the Taken's Theorem we can embed the time series we have available into a phase space where we do not have a temporal dependence between our data points anymore. This is important because the Statistical Learning Theory only guarantees the learning of an algorithm if the samples (rows or data instances) of our dataset are independent from each other.

In order to define this embedding, we need two hyperparameters named d and m which are the time delay and the embedding dimension respectively. The time delay defines how many data points, previously to the actual one, we will use on the embedding. The dimension will define the final shape of our dataset and how many axies there are in the phase space, each axis representing a time delay.

With those two parameters, given a time series $f(t)$, we will have its embedding as:

$$
\begin{matrix}
f(t) & f(t + d) & \dots & f(t + (m-1)d)\\
f(t+1) & f(t + d + 1) & \dots & f(t + (m-1)d + 1)\\
f(t+2) & f(t + d + 2) & \dots & f(t + (m-1)d) + 2)\\
\dots & \dots & \dots & \\
f(t+(m-1)) & f(t + d + (m -1)) & \dots &  f(t + (m-1)d + (m-1))
\end{matrix}
$$

The idea is that if the series is deterministic enough, then this embedding will generate an attractor on the phase space and we will able to forecast the time series.

In [5]:
# Extract time series of x variable
x = sol[1,:]

# Embedding parameters
m = 3   # embedding dimension
τ = 1   # time delay

N = length(x) - (m-1)*τ
X = zeros(m, N)
for i in 1:N
    for j in 1:m
        X[j,i] = x[i + (j-1)*τ]
    end
end

# Plot reconstructed phase space
scatter(X[1,:], X[2,:], X[3,:], markersize=1, xlabel="x", ylabel="y", zlabel="z", legend=false)

In [7]:
X

3×1262 Matrix{Float64}:
 1.0       0.999643  0.996105  0.969359  …  -4.60511   -4.77362   -6.70748
 0.999643  0.996105  0.969359  0.924204     -4.77362   -6.70748  -10.4
 0.996105  0.969359  0.924204  0.880046     -6.70748  -10.4      -12.1324

In [9]:
plot(X[1,:],X[2,:],X[3,:],xlabel="x",ylabel="y",zlabel="z",color=:)